# Setup LLM

In [ ]:
# set the models folder, HuggingFace will look into this folder and download the model if needed
%env HF_HOME=/kaggle/working
#Kaggle Notebooks

```Llama-3-8B-Instruct``` is a fine-tuned versions of the base model. We need to specific prompts:

In [3]:
# Adapted to only download one model as we do not have enough space for both on Kaggle
MODELS = {
    "llama3": "meta-llama/Meta-Llama-3-8B-Instruct",
}

TEMPLATES = {
    "llama3": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>",
}

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the HF_TOKEN
hf_token = os.getenv('HF_TOKEN')
print(f'HuggingFace Token: {hf_token}')

## Download the models (Only The first time)

1. Requested access [LLaMA 3](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct).
2. Download model with code below
3. It needs aboutire 15GB of storage space and 6-8 minutes

In [5]:
FIRST_TIME = False

if FIRST_TIME:
    from transformers import AutoModelForCausalLM, AutoTokenizer
    import torch
    
    def download_models(models):
        for model_name in models.values():
            # triggers download of the models
            AutoModelForCausalLM.from_pretrained(
                model_name,
                device_map="auto",
                torch_dtype=torch.float16
            )
            AutoTokenizer.from_pretrained(model_name)
    
    download_models(MODELS)

## Setup for prompting the model

Import the required libraries and classes.

In [6]:
import torch
import json

from typing import Tuple
from transformers import AutoModelForCausalLM, AutoTokenizer, BatchEncoding, PreTrainedTokenizer, PreTrainedModel

Functions for loading the models and generate responses.

In [7]:
def load_model(model_name: str, dtype) -> Tuple[PreTrainedModel, PreTrainedTokenizer]:
    torch_dtype = torch.float32
    if dtype == "bf16":
        torch_dtype = torch.bfloat16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch_dtype,

    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

def generate(
    model: PreTrainedModel,
    inputs: BatchEncoding,
    tokenizer: PreTrainedTokenizer,
    max_seq_length: int,
) -> str:
    output = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=max_seq_length,
        pad_token_id=tokenizer.eos_token_id,
    )
    return tokenizer.decode(
        output[0][len(inputs.input_ids[0]) :], skip_special_tokens=True
    )

Parameters and input for the generation.

In [8]:
model_name = "llama3"
chat_template = TEMPLATES[model_name]
model_name = MODELS[model_name]

dtype = "bf16"
#max_seq_length = 1024
max_seq_length = 2048

Load the model and tokenizer based on the parameters.

In [ ]:
model, tokenizer = load_model(model_name, dtype)

# Pipeline

## Segmentation

In [10]:
SEGMENTATION_PROMPT="""
Break the user input into multiple sentences based on the following intents:
- chicken_ordering, if the user wants to order chicken.
- drink_ordering, if the user wants to order a drink.
- dessert_ordering, if the user wants to order a dessert.
- appetizer_ordering, if the user wants to order an appetizer.
- table_reservation, if the user wants to reserve a table.
- request_information, if the user wants to know about something.
- out_of_domain, if nothing of the above fits.
Only provide the sentences, without the intent in list separated by commas, as follows:
["sentence1", "sentence2", ...]
Do not paraphrase and do not change the contents. Only provide the list.

For example:
INPUT: "Hi, I would like a roasted chicken with pesto sauce and potatoes. What drinks do you have?"
OUTPUT: ["Hi, I would like a roasted chicken with pesto sauce and potatoes.", "What drinks do you have?"]

INPUT: "Change the chicken size from medium to large please. I do not want ice in my drink."
OUTPUT: ["Change the chicken size from medium to large please.", "I do not want ice in my drink."]
"""

## NLU

In [11]:
NLU_PROMPT = """
You are a Natural Language Understanding component of a human dialogue system.
Identify the user intent from this list: [chicken_ordering, drink_ordering, dessert_ordering, table_reservation, request_information, out_of_domain].
- chicken_ordering, if the user wants to order chicken.
- drink_ordering, if the user wants to order a drink.
- dessert_ordering, if the user wants to order a dessert.
- appetizer_ordering, if the user wants to order an appetizer.
- table_reservation, if the user wants to reserve a table.
- request_information, if the user wants to know about something.
- out_of_domain, if nothing of the above fits.

If the intent is chicken_ordering, extract the following slot values from the user input:
- chicken_type, the type of chicken. Choose from: ["grilled", "roasted"].
- chicken_size, the size of the chicken. Choose from: ["small", "medium", "large"].
- chicken_bones, if the user wants the chicken with bones in it. Choose from: ["yes", "no"].
- sauce_type, the type of sauce that is served with the chicken. Choose from: ["mushroom", "pesto", "none"].
- side_dish, the side dish that is served with the chicken. Choose from: ["carrots", "smashed potatoes", "none"]. 
If no values are present in the user input you have to put "null" as the value. If the value is not among the candidates, then put "null". Output them in a JSON format. Only output the JSON file.
The JSON format is:
{
    "intent": "intent_value",
    "slots": {
        "slot1": "value1",
        "slot2": "value2",
        "slot3": "value3",
        "slot4": "value4",
        "slot5": "value5"
    }
}
For example:
INPUT: "I would like a medium chicken."
{
    'intent': 'chicken_ordering',
    'slots':
    {
        'chicken_type': "null",
        'chicken_size': 'medium',
        'chicken_bones': "null",
        'sauce_type': "null",
        'side_dish': "null"
    }
}

If the intent is drink_ordering, extract the following slot values from the user input:
- drink_type: the type of drink. Choose from: ["Coca Cola", "Fanta", "Sprite", "Water"].
- drink_size, the size of the drink. Choose from: ["small", "medium", "large"].
- ice: if the user wants ice in the drink. Choose from: ["yes", "no"].
If no values are present in the user input you have to put "null" as the value. If the value is not among the candidates, then put "null". Output them in a JSON format. Only output the JSON file.
For example: "Please add a Fanta as my drink."
{
    "intent": "drink_ordering",
    "slots": {
        "drink_type": "Fanta",
        "drink_size": "null",
        "ice": "null"
    }
}


If the intent is dessert_ordering, extract the following slot values from the user input:
- dessert_type, the type of dessert. Choose from: ["tiramisu", "ice cream", "apple crumble pie", "waffles", "smoothie"].
- extra_whipped_cream, if the user wants extra whipped cream. Choose from: ["yes", "no"].
If no values are present in the user input you have to put "null" as the value. If the value is not among the candidates, then put "null". Output them in a JSON format. Only output the JSON file.
For example: "I would like a tiramisu with whipped cream for dessert."
{
    "intent": "dessert_ordering",
    "slots": {
        "dessert_type": "tiramisu",
        "extra_whipped_cream": "yes"
    }
}


If the intent is appetizer_ordering, extract the following slot values from the user input:
- appetizer_type, the type of appetizer. Choose from: ["crisps", "simple salad", "deluxe salad", "tomato soup", "onion soup"].
- appetizer_moment, the time before the main course that the appetizer should arrive. Choose an integer based on the user input.
If no values are present in the user input you have to put "null" as the value. If the value is not among the candidates, then put "null". Output them in a JSON format. Only output the JSON file.
For example: "Please bring me a deluxe salad 8 minutes before the main course."
{
    "intent": "appetizer_ordering",
    "slots": {
        "appetizer_type": "deluxe salad",
        "appetizer_moment": "8"
    }
}


If the intent is table_reservation, extract the following slot values from the user input:
- table_type, the type of table the user wants to reserve. Choose from: ["normal", "business", "romantic"].
- table_size, the number of people that should fit at the table. Choose an integer based on the user input.
- sitting_equipment, if the user wants chairs or benches. Choose from: ["chair(s)", "bench(es)", "mixed chairs and benches", "does not matter"].
- birthday_surprise, if the user is celebrating someone's birthday at the dinner. Choose from ["yes", "no"].
If no values are present in the user input you have to put "null" as the value. If the value is not among the candidates, then put "null". Output them in a JSON format. Only output the JSON file.
For example: "I would like to reserve a table for 3 for a business meeting."
{
    "intent": "table_reservation",
    "slots": {
        "table_type": "business",
        "table_size": "3",
        "sitting_equipment": "null",
        "birthday_surprise": "null"
    }
}
For example: "We would like to reserve a table of size 5 for a birthday surprise."
{
    "intent": "table_reservation",
    "slots": {
        "table_type": "null",
        "table_size": "5",
        "sitting_equipment": "null",
        "birthday_surprise": "yes"
    }
}
For example: "No, we are not planning a birthday surprise."
{
    "intent": "table_reservation",
    "slots": {
        "table_type": "null",
        "table_size": "null",
        "sitting_equipment": "null",
        "birthday_surprise": "no"
    }
}

If the intent is request_information(entity):
Output it in a JSON format.
Only output the JSON file.
The JSON format is:
{
    "intent": "request_information(entity)"
}
For example: "What drinks do you have?"
{
    "intent": "request_information(drink_ordering)"
}
For example: "What chickens do you offer?"
{
    "intent": "request_information(chicken_ordering)"
}

If the intent is out_of_domain:
Output it in JSON format.
Only output the JSON file.
The JSON format is:
{
    "intent": "out_of_domain"
}
Examples of out_of_domain are:
- "I would like to order pizza."
- "I want to order a notebook."
"""

## DM

In [12]:
DM_PROMPT = """
You are a Dialogue Manager of a human dialogue system.
Given the outputs of the NLU component which is in a JSON format, you should only generate the next best actions from this list:
- request_details(slot), if a slot value is missing (None) by substituting slot with the missing slot name. The slot value is missing when None is the value.
- confirmation(intent), if all the values in the slots are filled. The slot is also filled when 'none' is the value.
- fallback_policy, if intent is out_of_domain.
- provide_information(entity), if the intent is request_information(entity).
- error_handling, if intent is set to error_handling.
Pay close attention that 'none' is not the same as None. 'none' is a filled field. None is an missing slot value.
Do not generate any other text.

For example:
INPUT: {'intent': 'chicken_ordering', 'slots': {'chicken_size': 'small', 'chicken_bones': 'yes', 'sauce_type': 'mushroom', 'side_dish': 'carrots', 'chicken_type': None}}
OUTPUT: request_details(chicken_type)

INPUT: {'intent': 'drink_ordering', 'slots': {'drink_size': 'small', 'drink_ice': 'no', 'drink_type': None}}
OUTPUT: request_details(drink_type)

INPUT: {'intent': 'dessert_ordering', 'slots': {'dessert_type': None, 'extra_whipped_cream': 'yes'}}
OUTPUT: request_details(dessert_type)

INPUT: {'intent': 'appetizer_ordering', 'slots': {'appetizer_moment': '2', 'appetizer_type': None}}
OUTPUT: request_details(appetizer_type)
"""

## NLG

In [13]:
NLG_PROMPT = """
You are the Natural Language Generation component of a human dialogue system. You must be very polite.
Given the next best action classified by the Dialogue Manager, you should only generate a lexicalized response for the user.
Prioritize provide_information before request_details.
Possible next best actions are:
- request_details(slot), generate an appropiate question to ask the users for the missing slot value.
- confirmation(intent), generate an appropiate confirmation message for the user intent. Fill in all the slot values.
- fallback_policy, generate an appropiate response to tell the user that the system is not designed to handle the request.
- error_handling(value), generate an appropiate response to tell the user that this value is not available.
- provide_information(entity), generate information about the entity, based on the options below.

Only propose options for slot values that are in the list below. The slot is the word before the ":"
- chicken_type, ["grilled", "roasted"].
- chicken_size, ["small", "medium", "large"].
- chicken_bones, ["yes", "no"].
- sauce_type, ["mushroom", "pesto", "none"].
- side_dish, ["carrots", "smashed potatoes", "none"].
- drink_type, ["Coca Cola", "Fanta", "Sprite", "Water"].
- drink_size, ["small", "medium", "large"].
- ice, ["yes", "no"].
- dessert_type, ["tiramisu", "ice cream", "apple crumble pie", "waffles", "smoothie"].
- extra_whipped_cream, ["yes", "no"].
- appetizer_type, ["crisps", "simple salad", "deluxe salad", "tomato soup", "onion soup"].
- appetizer_moment, the time before the main course that the appetizer should arrive as an integer
- table_type, ["normal", "business", "romantic"].
- table_size, the number of people that should fit at the table as an integer.
- sitting_equipment, ["chair(s)", "bench(es)", "mixed chairs and benches", "does not matter"].
- birthday_surprise, ["yes", "no"].
"""

In [14]:
NLG_PROMPT_CONFIRM = """
You are the Natural Language Generation component of a human dialogue system. You must be very polite.
Your task is to summarize the order given the NLU that you are given. You must mention all slots values from the NLU.
"""

## Database manager
This piece of code checks if all values are within specification

In [15]:
#Method 1: use a library:
#!pip install word2number
#from word2number import w2n
#text = "three"
#number = w2n.word_to_num(text)
#print(number)  # Output: 3

#Method 2: use a custom function that lies within reasonable specification
def word_to_num(word):
    words_to_numbers = {
        "one": 1,
        "two": 2,
        "three": 3,
        "four": 4,
        "five": 5,
        "six": 6,
        "seven": 7,
        "eight": 8,
        "nine": 9,
        "ten": 10,
        "eleven": 11,
        "twelve": 12,
        "thirteen": 13,
        "fourteen": 14,
        "fifteen": 15,
        "sixteen": 16,
        "seventeen": 17,
        "eighteen": 18,
        "nineteen": 19,
        "twenty": 20
    }
    return words_to_numbers.get(word.lower(), None)

In [16]:
def validate_slots(data):
    # Define valid options for each slot
    valid_options = {
        'chicken_type': ["grilled", "roasted", "null"],
        'chicken_size': ["small", "medium", "large", "null"],
        'chicken_bones': ["yes", "no", "null"],
        'sauce_type': ["mushroom", "pesto", "none", "null"],
        'side_dish': ["carrots", "smashed potatoes", "none", "null"],
        'drink_type': ["Cola", "Coca Cola", "Fanta", "Sprite", "Water", "null"],
        'drink_size': ["small", "medium", "large", "null"],
        'ice': ["yes", "no", "null"],
        'dessert_type': ["tiramisu", "ice cream", "apple crumble pie", "waffles", "smoothie", "null"],
        'extra_whipped_cream': ["yes", "no", "null"],
        'appetizer_type': ["crisps", "simple salad", "deluxe salad", "tomato soup", "onion soup", "null"],
        'appetizer_moment': int, # Must be an integer
        'table_type': ["normal", "business", "romantic", "null"],
        'table_size': int,  # Must be an integer
        'sitting_equipment': ["chair(s)", "bench(es)", "mixed chairs and benches", "does not matter", "null"],
        'birthday_surprise': ["yes", "no", "null"]
    }

    # Define valid intents
    valid_base_intents = [
        "chicken_ordering",
        "drink_ordering",
        "dessert_ordering",
        "table_reservation"
    ]
    valid_intents = valid_base_intents + [f"request_information({intent})" for intent in valid_base_intents]
    valid_intents = valid_intents + ["out_of_domain"]
    
    # Check intent
    intent = data.get('intent')
    if intent not in valid_intents:
        return False, None

    # If intent is a request_information intent, ensure no slots are provided
    if intent.startswith("request_information("):
        base_intent = intent[len("request_information("):-1]
        if base_intent not in valid_base_intents or data.get('slots'):
            return False, None
        return True, None

    # Check slots
    slots = data.get('slots', {})
    for slot, value in slots.items():
        # Skip validation if the value is None
        if value is None:
            continue

        # Check if slot is in valid options
        if slot not in valid_options:
            return False, None

        # Validate value based on its type or list of valid values
        valid_values = valid_options[slot]
        if isinstance(valid_values, list):
            if value not in valid_values:
                return False, value
        elif valid_values == int:
            if value.isdigit(): #E.g."50".isdigit() = True
                continue
            value = word_to_num(value)
            if not isinstance(value, int):
                return False, value

    return True, None

## DST Manager

In [17]:
import re

class DialogueStateTracker:
    def __init__(self):
        self.previous_state = {"NLU": None, "DM": None}
        self.order_confirmed = False
        self.not_working_value = None

    def update_nlu_state(self, nlu_out: str):
        nlu_out_dict = dict(eval(self.find_dictionary(nlu_out)))
        nlu_out_dict = self.change_null(nlu_out_dict)

        # Check the validity of the slots
        valid_bool, value = validate_slots(nlu_out_dict)

        
        
        if valid_bool == False:
            print("Error handling active")
            self.update_dm_state("error_handling")
            if value is not None:
                self.update_not_working_value(value)
        else:
            if self.previous_state["NLU"] is None:
                self.previous_state["NLU"] = nlu_out_dict
            else: 
                self.previous_state["NLU"] = self.deepmerge_dicts(self.previous_state["NLU"], nlu_out_dict)
        return self.previous_state

    def update_not_working_value(self, value):
        self.not_working_value = value
        
    def get_not_working_value(self):
        return self.not_working_value
        
    def error_handling(self):
        self.previous_state["NLU"] = {"intent": "error_handling"}
        return self.previous_state

    def check_slot_values_null(self):
        if dst.get_state()["NLU"] == None:
            return True #e.g. by error-handling
        if 'slots' not in list(dst.get_state()["NLU"].keys()):
            return True #e.g. for request_information
        else:
            return None in list(dst.get_state()["NLU"]['slots'].values())

    def update_dm_state(self, dm_out: str):
        self.previous_state["DM"] = dm_out
        return self.previous_state
    
    def get_state(self):
        return self.previous_state
    
    def order_confirmation(self):
        self.order_confirmed = True

    def get_confirmation_status(self):
        return self.order_confirmed
    
    def change_null(self, dictionary: dict) -> dict:
        """
        Recursively replaces all occurrences of the string "null" with Python's None in the given dictionary.

        Args:
            dictionary (dict): The input dictionary.

        Returns:
            dict: A dictionary with "null" replaced by None.
        """
        for key, value in dictionary.items():
            if value == "null":
                dictionary[key] = None
            elif isinstance(value, dict):  # If the value is a dictionary, recurse.
                dictionary[key] = self.change_null(value)
        return dictionary
    
    def deepmerge_dicts(self, d1: dict, d2: dict) -> dict:
        merged = {}
        for key in set(d1) | set(d2):
            if key in d1 and key in d2:
                if isinstance(d1[key], dict) and isinstance(d2[key], dict):
                    merged[key] = self.deepmerge_dicts(d1[key], d2[key])
                else:
                    #Take latest value, in case user wants to change option.
                    if d2[key] is None:
                        merged[key] = d1[key]
                    else:
                        merged[key] = d2[key]
            elif key in d1:
                merged[key] = d1[key]
            else:
                merged[key] = d2[key]
        return merged
    
    def find_dictionary(self, text: str) -> str:
        opening_bracket = [(match.start(), match.group()) for match in re.finditer(r'[{]', text)]
        opening_index = opening_bracket[0][0]

        closing_bracket = [(match.start(), match.group()) for match in re.finditer(r'[}]', text)]
        closing_index = closing_bracket[-1][0]

        dictionary_from_llm = text[opening_index:closing_index+1]
        return dictionary_from_llm

In [29]:
def postprocess_nlg(nlg_output):
    nlg_output = nlg_output.replace('"', '')
    nlg_output = nlg_output.replace('\n', '')
    if ":" in nlg_output:
        nlg_output = nlg_output.split(':', 1)[1].strip()
    return nlg_output

def postprocess_nlg_summary(nlg_output):
    if ":" in nlg_output:
        nlg_output = nlg_output.split(':', 1)[1].strip()
    return nlg_output
    
def postprocess_dm(dm_output):
    if "\n" in dm_output:
        dm_output = dm_output.strip("\n")
    if "'" in dm_output:
        dm_output = dm_output.replace("'", "")
    if '"' in dm_output:
        dm_output = dm_output.replace('"', '')
    return dm_output

def use_LLM(PROMPT, text):
    input_template = chat_template.format(PROMPT, text)
    output_tokenizer = tokenizer(input_template, return_tensors="pt").to(model.device)
    output = generate(model, output_tokenizer, tokenizer, max_seq_length)
    return output

def generate_goodybye(DSTs):
    intents = []
    for dst in DSTs:
        intent = dst.get_state()["NLU"]["intent"] #get the state
        #check if it does not start with request_information
        if "request_information" not in intent:
            typ = intent.split("_")[0] #get the thing before the underscore
            intents.append(typ)
    out = "Thanks for ordering "
    for typ in intents:
        if len(intents) == 1: #if there is only one order, just output the name
            out = out + typ
        else:
            if typ == intents[-1]: #if we are at the last element, use an extra 'and'
                out = out + "and " + typ
            else:
                out = out + typ + ", "
    out = out + "\nGoodbye!"
    return out

# Full pipeline - NEW V4

In [32]:
DEBUG = False

In [ ]:
print("Welcome to chicken restaurant ROASTED. How can I help you today?")

STOP_FLAGS = []
start_conv = True
DSTs = []
confirmed_orders_dsts = []

while len(DSTs) == 0 or not all([dst.get_confirmation_status() for dst in DSTs]):
    user_input = input()
    outputs = []
    
    #SEGMENT
    i = 0
    while i<5: #Max 5 attemtps
        try:
            segment_output = use_LLM(SEGMENTATION_PROMPT, user_input)
            # Try to decode the JSON string
            segment_output = json.loads(segment_output)
            segmentation_correct = True
            break  # Exit the loop if decoding is successful
        except json.JSONDecodeError as e:
            i = i + 1
            if DEBUG: print(f"JSONDecodeError while segmenting, try again: {e}")

    if segmentation_correct == False: #if the segmentation failed for more than 5 times, we fallback
        nlg_output = use_LLM(NLG_PROMPT, "fallback_policy")
        #POSTPROCESS
        nlg_output_postprocessed = postprocess_nlg(nlg_output)
        if DEBUG: print(nlg_output_postprocessed)
    else: #else, we continue with the putting the segmentation to the NLU
        for i in range(len(segment_output)):
            segment = segment_output[i]
            if DEBUG: print(f"SEGMENT: {segment}")
            
            #Check if there is an empty segment, if so do nothing
            segment = segment.replace("\n", "")
            if len(segment) == 0:
                continue
            else:
                nlu_output = use_LLM(NLU_PROMPT, segment)
                if DEBUG: print(f"NLU_OUTPUT LLAMA: {nlu_output}")
    
            #Check if DST has such a intent:
            #if so continue there
            #else add a new DST
            if len(DSTs) == 0: #If there is not a single one in the list, then create one
                dst = DialogueStateTracker()
                DSTs.append(dst)
                #print("INIT FIRST ONE")
            else: #Else, search through the DSTs if there is one with the same intent, take it else create new
                #print("FINDING DST")
                found_flag = False
                for dst in DSTs:
                    if dst.get_state()["NLU"] == None:
                        continue
                    a = str(dst.get_state()["NLU"]["intent"]) in str(nlu_output)
                    b = not dst.get_confirmation_status()
                    #a = str(nlu_output).__contains__(str(dst.get_state()["NLU"]["intent"]))
                    if a and b:# and not b:# and not "provide_information" in str(dst.get_state()["DM"]):
                        #print("FOUND DST:", DSTs.index(dst))
                        dst = dst
                        found_flag = True
                        break #we found the corresponding dst
                if found_flag == False: #there is no one in there yet, so we make one for the intent
                    #print("CREATING NEW DST")
                    dst = DialogueStateTracker()
                    DSTs.append(dst)
            
            #Try to update the NLU, assuming the correct format.
            try:
                dst.update_nlu_state(nlu_out=nlu_output)
            except Exception as e:
                if DEBUG: 
                    print(f"An unexpected error occurred: {e} in segment {segment}.")
                    print("Activating error handling.")
            nlu_in_dst = str(dst.get_state()["NLU"])
            if DEBUG:
                print("NLU:")
                print(nlu_in_dst)
                print()

        # Now, we have processed all segments to the NLU, we continue with the DSTs
        for dst in DSTs:
            if DEBUG:
                print("NLU_state", str(dst.get_state()["NLU"]))
                print("DM_state", str(dst.get_state()["DM"]))
            if (dst.get_state()["NLU"]) == None:
                if str(dst.get_state()["DM"]) == "error_handling":
                    if DEBUG: print("DST found with error handling, setting DM_output to error_handling")
                    dm_output = "error_handling"
                else:
                    continue
            elif dst.get_confirmation_status():
                continue
            elif dst.check_slot_values_null() == False:
                dm_output = f"confirmation('{dst.get_state()['NLU']['intent']}')"
            else:
                dm_output = use_LLM(DM_PROMPT, str(dst.get_state()["NLU"]))
                dm_output = postprocess_dm(dm_output) #overwrite to dm_output as that's what we use
            dst.update_dm_state(dm_out=dm_output)
            dm_in_dst = str(dst.get_state()["DM"])
            if DEBUG: print("DM:", dm_in_dst)

            # If all values are filled and we are not handling an error
            if str(dst.get_state()["DM"]) != "error_handling" and dst.check_slot_values_null() == False:
                #Summarize order
                nlg_output = use_LLM(NLG_PROMPT_CONFIRM, str(dst.get_state()))
                nlg_output_postprocessed = postprocess_nlg_summary(nlg_output)
                print(nlg_output_postprocessed)

                #User confirmation
                response_not_recognized = True
                while response_not_recognized:
                    user_input = input("Confirmation with YES or NO:")
                    user_input = str(user_input).lower()
                    if "yes" in user_input or "y" in user_input:
                        dst.order_confirmation()
                        confirmed_orders_dsts.append(dst)
                        response_not_recognized = False
                        #print('Thanks for your order.')
                        inte = str(dst.get_state()["NLU"]["intent"])
                        #c_message = "Your order has been placed"
                        c_message = f"Your {inte.replace('_', ' ')} has been placed!"
                        outputs.append(c_message)
                    elif "no" in user_input or "n" in user_input:
                        response_not_recognized = False
                        print("Please specify what you want to change in your next turn.")
                        inte = str(dst.get_state()["NLU"]["intent"])
                        changing_message = f"What would you like to change in your {inte.replace('_', ' ')}?"
                        outputs.append(changing_message)
                    else:
                        print("Sorry, I did not understand. Do you want to confirm your order (YES) or (NO)?")
            elif str(dst.get_state()["DM"]) == "error_handling":
                val = dst.get_not_working_value()
                if val == None:
                    print("ERROR, WTF AM I DOING???") #if this happens, check the value in nlu_update attempt with flag
                dm_in_dst = f"error_handling({val})"
                nlg_output = use_LLM(NLG_PROMPT, dm_in_dst)
                nlg_output_postprocess = postprocess_nlg(nlg_output) #Postprocess
                outputs.append(nlg_output_postprocess) #Append to output list
                dm_output = None #reset dm_output
                dst.update_dm_state(dm_out=dm_output)
            else:
                nlg_output = use_LLM(NLG_PROMPT, dm_in_dst)
                nlg_output_postprocess = postprocess_nlg(nlg_output) #Postprocess
                outputs.append(nlg_output_postprocess) #Append to output list    
            
            #As this is an information request and not for details, we can directly set the confirmation to True.
            #This way we don't have to process it again next time
            if dst.get_state()["NLU"] is not None and "request_information" in str(dst.get_state()["NLU"]["intent"]):
                dst.order_confirmation()
            #if str(dst.get_state()["DM"]) == "error_handling":
            #    dst.order_confirmation()
        print()
        if len(outputs) == 0:
            OUT = ""
        elif len(outputs) == 1:
            OUT = outputs[0]
        elif len(outputs) > 1:
            OUT = " ".join(outputs)
        elif all([dst.get_confirmation_status() for dst in DSTs]):
            OUT = "Goodbye!"
        else:
            print("Unknown problem with outputs")
        print(OUT)
        print("="*50)
print(generate_goodybye(DSTs)) #Goodbye screen with string injection

I want to drink a lemon soda.

I want to drink a lemon soda.

I want my drink to be of medium size without ice.

Please give me a lemon soda.

```I apologize, but I didn't quite catch what you said. Could you please repeat or rephrase your request? I'm here to help and want to make sure I understand correctly.```

### Examples
#### Example 1 - mixed (chicken, drink)
- Hi, I would like a roasted chicken with pesto sauce and smashed potatoes. What drinks do you have?
- I would like a medium sized chicken without bones. I would like a small Coca Cola.
- Accept chicken
- Yes, I would like ice in my drink.
- Accept drink

#### Example 2.1 - mixed (chicken, drink, alternating)
- Hi, I would like a roasted chicken with pesto sauce and smashed potatoes. What drinks do you have?
- I would like a medium sized chicken without bones. I would like a small Coca Cola.
- Reject chicken
- Change the chicken size from medium to large please. I do not want ice in my drink. (NLU fails on 1st segment)
- Accept drink
- I want a large chicken instead of a medium one. (NLU works)
- Accept chicken

#### Example 2.2 - mixed (chicken, drink, alternating)
- Hi, I would like a roasted chicken with pesto sauce and smashed potatoes. What drinks do you have?
- I would like a medium sized chicken without bones. I would like a small Coca Cola.
- Reject chicken
- Change the chicken size from medium to large please. I do not want ice in my drink. (NLU works)
- Accept both

#### Example 3 - mixed later (table, chicken)
- I would like to reserve a table for 4 persons.
- No, we are not planning a birthday surprise.
- I prefer a normal table. What chickens do you offer?
- I want a mix of seats at the table.

#### Example 4 - error handling (chicken, drink not from list)
- I would like my chicken to be of medium size. For my drink, I'd like a lemon soda.
- Please serve the chicken grilled with mushroom sauce. I want to drink a lemon soda.
- The chicken without bones please.
- Please serve my chicken with no side dish.
- Accept chicken
- I want a lemon soda. (NLU does not pick option)
- I would really like a lemon soda please. (Nope)
- Please, I really need a lemon soda. (Yay, error handling :) )

#### Example 5 - error handling (lemon soda)
- I want a lemon soda. (sometimes accepts)
- I would really like a lemon soda please. (sometimes accepts)
- Please, I really need a lemon soda.


Keeps on asking for drink type:
SEGMENT: I want a lemon soda.
NLU_OUTPUT LLAMA: 

{
    "intent": "drink_ordering",
    "slots": {
        "drink_type": "Lemon Soda",
        "drink_size": "null",
        "ice": "null"
    }
}
Error handling active
NLU:
{'intent': 'drink_ordering', 'slots': {'drink_type': None, 'drink_size': None, 'ice': None}}

DM:
request_details('drink_type')

What type of drink would you like to have with your meal? We have Coca Cola, Fanta, or Sprite available. Which one would you prefer?


- I will settle for a Sprite

Check this sentence:
Hi, I would like a roasted chicken with pesto sauce and broccoli.
or
For my drink, I'd like a lemon soda

In [20]:
#Breaks:
#I would like my chicken to be of medium size. For my drink, I'd like a lemon soda. becaus it picks lemon soda as drink and does not return error

In [21]:
#I would like a medium chicken with pesto sauce. What drinks do you have?
#I would like a roasted chicken. Please add a Fanta as my drink.
#I would like a boneless chicken. I would prefer a medium-sized drink.